https://gist.github.com/YohanObadia/b310793cd22a4427faaadd9c381a5850

In [8]:
import numpy as np
import pandas as pd
from collections import defaultdict
from scipy.stats import hmean
from scipy.spatial.distance import cdist
from scipy import stats
import numbers

In [11]:
data = pd.read_csv("C:\\Users\\Lourdes\\Downloads\\DataFullRemoved.csv")

In [12]:
data.head()

,Date,520_Madison_Board_A-1_(Mechanical),_520_Madison_Board_A-2_(Mechanical),_520_Madison_Board_B-1_(Mechanical),_520_Madison_Board_B-2_(Mechanical),_520_Madison_Board_C-1_(Mechanical),_520_Madison_Total_Building_Electric_(Mechanical),_520_Madison_Board_A_(Tenant),_520_Madison_Board_B_(Tenant),_520_Madison_Board_C_(Tenant),...,_520_Madison_Board_A_(Con-Ed_8095353),_520_Madison_Board_B_(Con-Ed_8095349),_520_Madison_Board_C_(Con-Ed_8095356),_520_Madison_Tenant_Board_C_(Con-Ed_7754273),_520_Madison_Total_Building_Electric_(Con-Ed),_NYC_Hourly_Humidity,_NYC_Hourly_Temperature,_NYC_Hourly_Wetbulb_Temperature,Month,Day
0,1/1/2015,32.947917,6.666667,13.020833,6.052083,6.520833,65.208333,102.052083,86.093750,12.020833,...,141.666667,105.166667,18.541667,118.645833,384.020833,9.689583,32.920833,26.329250,1.0,5.0
1,1/2/2015,43.979167,9.906250,16.229167,8.760417,24.843750,103.718750,113.156250,92.864583,12.989583,...,167.041667,117.854167,37.833333,129.208333,451.937500,16.595833,38.485417,31.265354,1.0,6.0
2,1/3/2015,28.552083,6.614583,16.729167,6.135417,11.895833,69.927083,94.208333,72.552083,10.062500,...,129.375000,95.416667,21.958333,110.625000,357.375000,26.170833,36.247917,32.834812,1.0,7.0
3,1/4/2015,32.093750,NaN,7.531250,6.229167,8.770833,61.072917,96.708333,72.989583,8.416667,...,135.250000,86.750000,17.187500,112.395833,351.583333,44.164583,48.420833,46.808687,1.0,1.0
4,1/5/2015,59.531250,11.916667,4.302083,10.781250,30.145833,116.677083,124.406250,102.000000,13.020833,...,195.854167,117.083333,43.166667,143.729167,499.833333,10.939583,35.618750,28.464646,1.0,2.0


In [13]:
def weighted_hamming(data):
    categories_dist = []
    
    for category in data:
        X = pd.get_dummies(data[category])
        X_mean = X * X.mean()
        X_dot = X_mean.dot(X.transpose())
        X_np = np.asarray(X_dot.reaplace(0,1,inplace=False))
    categories_dist = np.array(categories_dist)
    distances = hmean(categories_dist, axis=0)
    return distances

In [20]:
def distance_matrix(data, numeric_distance = "euclidean", categorical_distance = "jaccard"):
    possible_continious_distances = ["euclidean", "cityblock"]
    possible_binary_distances = ["euclidean", "jaccard", "hamming", "wighted-hamming"]
    number_of_variables = data.shape[1]
    number_of_observations = data.shape[0]
    
    #Get the number of each attribute (numeric or categorical)
    is_numeric = [all(isinstance(n, numbers.Number) for n in data.iloc[:, i]) for i, x in enumerate(data)]
    is_all_numeric = sum(is_numeric) == len(is_numeric)
    is_all_categorical = sum(is_numeric) == 0 
    is_mixed_type = not is_all_categorical and not is_all_numeric
    
    #check the content of the distance parameter
    if numeric_distance not in possible_continuous_distances:
        print"The continuous distance " + numeric_distance + "is not supported."
        return None
    elif categorical_distance not in possible_binary_distances:
        print "The binary distance" + categorical_distance + " is not supported."
        return None
    
    #separate the data frame into categorical and numeric attriubutes and normalize numeric data
    if is_mixed_type:
        number_of_numeric_var = sum(is_numeric)
        number_of_categorical_var = number_of_variables - number_of_numeric_var
        data_numeric = data.iloc[:, is_numeric]
        data_numeric = (data_numeric - data_numeric.mean()) / (data_numeric.max() - data_numeric.min())
        data_categorical = data.iloc[:, [not x for x in is_numeric]]
        
    # Replace missing values with column mean for numeric values and mode for categorical ones. With the mode, it
    # triggers a warning: "SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame"
    # but the value are properly replaced
    if is_mixed_type:
        data_numeric.fillna(data_numeric.mean(), inplace=True)
        for x in data_categorical:
            data_categorical[x].fillna(data_categorical[x].mode()[0], inplace=True)
    elif is_all_numeric:
        data.fillna(data.mean(), inplace=True)
    else:
        for x in data:
            data[x].fillna(data[x].mode()[0], inplace=True)
            
    # "Dummifies" categorical variables in place
    if not is_all_numeric and not (categorical_distance == 'hamming' or categorical_distance == 'weighted-hamming'):
        if is_mixed_type:
            data_categorical = pd.get_dummies(data_categorical)
        else:
            data = pd.get_dummies(data)
    elif not is_all_numeric and categorical_distance == 'hamming':
        if is_mixed_type:
            data_categorical = pd.DataFrame([pd.factorize(data_categorical[x])[0] for x in data_categorical]).transpose()
        else:
            data = pd.DataFrame([pd.factorize(data[x])[0] for x in data]).transpose()
            
    if is_all_numeric:
        result_matrix = cdist(data, data, metric=numeric_distance)
    elif is_all_categorical:
        if categorical_distance == "weighted-hamming":
            result_matrix = weighted_hamming(data)
        else:
            result_matrix = cdist(data, data, metric=categorical_distance)
    else:
        result_numeric = cdist(data_numeric, data_numeric, metric=numeric_distance)
        if categorical_distance == "weighted-hamming":
            result_categorical = weighted_hamming(data_categorical)
        else:
            result_categorical = cdist(data_categorical, data_categorical, metric=categorical_distance)
        result_matrix = np.array([[1.0*(result_numeric[i, j] * number_of_numeric_var + result_categorical[i, j] *
                               number_of_categorical_var) / number_of_variables for j in range(number_of_observations)] for i in range(number_of_observations)])
    
    # Fill the diagonal with NaN values
    np.fill_diagonal(result_matrix, np.nan)

    return pd.DataFrame(result_matrix)

In [15]:
def knn_impute(target=data['Month'], attributes=data.drop(['Day'], 1), k_neighbors=4, aggregation_method="mean", numeric_distance="euclidean",
               categorical_distance="jaccard", missing_neighbors_threshold = 0.5):
    # Get useful variables
    possible_aggregation_method = ["mean", "median", "mode"]
    number_observations = len(target)
    is_target_numeric = all(isinstance(n, numbers.Number) for n in target)
    
    # Check for possible errors
    if number_observations < 3:
        print "Not enough observations."
        return None
    if attributes.shape[0] != number_observations:
        print "The number of observations in the attributes variable is not matching the target variable length."
        return None
    if k_neighbors > number_observations or k_neighbors < 1:
        print "The range of the number of neighbors is incorrect."
        return None
    if aggregation_method not in possible_aggregation_method:
        print "The aggregation method is incorrect."
        return None
    if not is_target_numeric and aggregation_method != "mode":
        print "The only method allowed for categorical target variable is the mode."
        return None

    # Make sure the data are in the right format
    target = pd.DataFrame(target)
    attributes = pd.DataFrame(attributes)

    # Get the distance matrix and check whether no error was triggered when computing it
    distances = distance_matrix(attributes, numeric_distance, categorical_distance)
    if distances is None:
        return None

    # Get the closest points and compute the correct aggregation method
    for i, value in enumerate(target.iloc[:, 0]):
        if pd.isnull(value):
            order = distances.iloc[i,:].values.argsort()[:k_neighbors]
            closest_to_target = target.iloc[order, :]
            missing_neighbors = [x for x  in closest_to_target.isnull().iloc[:, 0]]
            # Compute the right aggregation method if at least more than 50% of the closest neighbors are not missing
            if sum(missing_neighbors) >= missing_neighbors_threshold * k_neighbors:
                continue
            elif aggregation_method == "mean":
                target.iloc[i] = np.ma.mean(np.ma.masked_array(closest_to_target,np.isnan(closest_to_target)))
            elif aggregation_method == "median":
                target.iloc[i] = np.ma.median(np.ma.masked_array(closest_to_target,np.isnan(closest_to_target)))
            else:
                target.iloc[i] = stats.mode(closest_to_target, nan_policy='omit')[0][0]

    return target

In [19]:
result_matrix.head()

NameError: name 'result_matrix' is not defined